- `coalesce` and `repartition` are functions on top of the dataframe. Do not get confused between **coalesce** on DataFrame and the coalsece function available to deal with null in a given col
- `coalesce` is typically used to **reduce number of partitions** to deal with as part of downstream processing
- `repartition` is used to reshuffle the data to **higher or lower number of partitions** to deal with as part of downstream processing
- Make sure to use a cluster with higher configuration, if you would like to run and experience by your self.

In [0]:
df = spark.read.csv('dbfs:/databricks-datasets/asa/airlines', header=True)

In [0]:
help(df.coalesce)

Help on method coalesce in module pyspark.sql.dataframe:

coalesce(numPartitions: int) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` that has exactly `numPartitions` partitions.
    
    Similar to coalesce defined on an :class:`RDD`, this operation results in a
    narrow dependency, e.g. if you go from 1000 partitions to 100 partitions,
    there will not be a shuffle, instead each of the 100 new partitions will
    claim 10 of the current partitions. If a larger number of partitions is requested,
    it will stay at the current number of partitions.
    
    However, if you're doing a drastic coalesce, e.g. to numPartitions = 1,
    this may result in your computation taking place on fewer nodes than
    you like (e.g. one node in the case of numPartitions = 1). To avoid this,
    you can call repartition(). This will add a shuffle step, but means the
    current upstream partitions will be executed in parallel (per whatever
 

In [0]:
help(df.repartition)

Help on method repartition in module pyspark.sql.dataframe:

repartition(numPartitions: Union[int, ForwardRef('ColumnOrName')], *cols: 'ColumnOrName') -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` partitioned by the given partitioning expressions. The
    resulting :class:`DataFrame` is hash partitioned.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    numPartitions : int
        can be an int to specify the target number of partitions or a Column.
        If it is a Column, it will be used as the first partitioning column. If not specified,
        the default number of partitions is used.
    cols : str or :class:`Column`
        partitioning columns.
    
        .. versionchanged:: 1.6.0
           Added optional arguments to specify the partitioning columns. Also made numPartitions
           optional if partitioning columns are specifi

In [0]:
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1988|    1|         9|        6|   1348|      1331|   1458|      1435|           PI|      942

- `repartition` incurs shuffling and it takes time as data has to be shuffled to newer number of partitions
- Also you can `repartition` the DataFrame based on specified columns
- `coalesce` does not incur shuffling
- We use `coalesce` quite often before writing the data to fewer number of files

In [0]:
df = spark.read.csv('dbfs:/databricks-datasets/asa/airlines', header=True,inferSchema=True)

In [0]:
dbutils.fs.ls('dbfs:/databricks-datasets/asa/airlines')

[FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1987.csv', name='1987.csv', size=127162942, modificationTime=1459744248000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1988.csv', name='1988.csv', size=501039472, modificationTime=1459744260000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1989.csv', name='1989.csv', size=486518821, modificationTime=1459744335000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1990.csv', name='1990.csv', size=509194687, modificationTime=1459744384000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1991.csv', name='1991.csv', size=491210093, modificationTime=1459744438000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1992.csv', name='1992.csv', size=492313731, modificationTime=1459744493000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1993.csv', name='1993.csv', size=490753652, modificationTime=1459744545000),
 FileInfo(path='dbfs:/databricks-datasets/asa/airlines/1994.csv', name='1994

In [0]:
df.rdd.getNumPartitions()

93

In [0]:
#coalescing the dataframe to 16

df.coalesce(16).rdd.getNumPartitions()

16

In [0]:
# not effective as coalesce can be used to reduce the number of partitions. Faster as no shuffling is involved
df.coalesce(186).rdd.getNumPartitions()

93

In [0]:
df.repartition(16).rdd.getNumPartitions()

16

In [0]:
df.repartition(186, 'Year', 'Month').rdd.getNumPartitions()